In [13]:
import requests
from io import BytesIO
import gzip
import pandas as pd

Getting the GZIP containing the dataset file

In [11]:
# URL of the GZIP containing the dataset file
URL_IMDB_DATASET_GZIP = "http://127.0.0.1:5500/title.basics.tsv.gz" #"https://datasets.imdbws.com/title.basics.tsv.gz"

# Requesting the GZIP
gzip_response = requests.get(URL_IMDB_DATASET_GZIP)

b'\x1f\x8b\x08\x08\xb0\xc2\xf4c\x00\x03data.tsv\x00\xac\\[o\xe48v~\xae\xfc\n\xee\x93\x13\xc0=p\x95\xef\x9b\'\xdbe\xf7e\xedn\xc3\xe5\xee\x9e\t\xf2\xc2\x92XU\x1cK\xa2\x86\x94lW\xff\x83\xfd\x17;\xb9`\xda\x0bL\x80 A\x80$\xd8\'\xfd\xb1|\x87*QR\x95\xa4Rgv03.\x1dJ\x87"y'


In [18]:
with gzip.open(BytesIO(gzip_response.content), 'rt') as gzip_file:
    # Creating Dataframe from byte object
    imdb = pd.read_csv(gzip_file, sep="\t", na_values=r'\N', \
        dtype={ \
            "tcost": object, \
            "titleType": object, \
            "primaryTitle": object, \
            "originalTitle": object, \
            "isAdult": object, \
            "startYear": 'Int64', \
            "endYear": object, \
            "runtimeMinutes": object, \
            "genres": object, \
        } \
    )

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"


Filtering and cleaning the dataset

In [19]:
# Filtering only movies
imdb = imdb.loc[imdb.titleType == "movie"]

# Dropping any movie with no year of release
imdb = imdb.dropna(subset="startYear")

# Dropping unnecessary columns
imdb.drop(
    labels=['tconst', 'titleType', 'originalTitle', 'isAdult', 'endYear'],
    axis=1,
    inplace=True
)

Writing to .TSV file

In [8]:
imdb.to_csv("imdb.tsv", sep="\t")